In [7]:
# Import des bibliothèques nécessaires
import pandas as pd
import plotly.express as px
import dash
from dash import dcc, html, Input, Output
import dash_bootstrap_components as dbc
import datetime

# Charger les données
df = pd.read_csv('./Projet/supermarket_sales.csv')

# Convertir la colonne Date en format datetime
df['Date'] = pd.to_datetime(df['Date'])

# Créer une colonne pour la semaine
df['Week'] = df['Date'].dt.isocalendar().week
df['Year_Week'] = df['Date'].dt.strftime('%Y-%V')

# Initialiser l'application Dash avec un thème Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], title="Tableau de Bord des Ventes de Supermarché")

# Définir les options pour les filtres
gender_options = [{'label': 'Tous', 'value': 'All'}] + [{'label': gender, 'value': gender} for gender in df['Gender'].unique()]
city_options = [{'label': 'Toutes', 'value': 'All'}] + [{'label': city, 'value': city} for city in df['City'].unique()]

#-----------Maquette de l'application-----------#

app.layout = dbc.Container([
    dbc.Row([
        dbc.Col([
            html.H1("Tableau de Bord des Ventes de Supermarché", className="text-center my-4")
        ])
    ], style={"background-color": "lightblue", "color": "black", "padding": "20px", "margin-bottom": "30px"}),
    
    # Filtres
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Filtres", className="text-center")),
                dbc.CardBody([
                    dbc.Row([
                        dbc.Col([
                            dbc.Label("Sexe"),
                            dcc.Dropdown(
                                id='gender-filter',
                                options=gender_options,
                                value='All',
                                clearable=False
                            ),
                        ], md=6),
                        
                        dbc.Col([
                            dbc.Label("Ville"),
                            dcc.Dropdown(
                                id='city-filter',
                                options=city_options,
                                value='All',
                                clearable=False
                            ),
                        ], md=6),
                    ])
                ])
            ], className="filters-card")
        ])
    ], className="mb-4"),
    
    # Indicateurs dans des cartes
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Montant total des achats", className="text-center")),
                dbc.CardBody([
                    html.H3(id='total-amount', className="text-center text-primary")
                ],style={"background-color": "#e8f4f8"})
            ], className="h-100")
        ], md=6),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Nombre total d'achats", className="text-center")),
                dbc.CardBody([
                    html.H3(id='total-purchases', className="text-center text-primary")
                ],style={"background-color": "#e8f4f8"})
            ], className="h-100")
        ], md=6),
    ], className="mb-4"),
    
    # Histogramme
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Répartition des montants totaux des achats", className="text-center")),
                dbc.CardBody([
                    dcc.Graph(id='histogram-chart')
                ])
            ])
        ], md=12),
    ], className="mb-4"),
    
    # Graphiques (pie et line)
    dbc.Row([
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Répartition des catégories de produits", className="text-center")),
                dbc.CardBody([
                    dcc.Graph(id='pie-chart')
                ])
            ], className="h-100")
        ], md=6),
        
        dbc.Col([
            dbc.Card([
                dbc.CardHeader(html.H4("Évolution des achats par semaine", className="text-center")),
                dbc.CardBody([
                    dcc.Graph(id='line-chart')
                ])
            ], className="h-100")
        ], md=6),
    ]),
    
], fluid=True, className="p-4")

# Fonctions pour filtrer les données
def filter_data(gender, city):
    filtered_df = df.copy()
    
    if gender != 'All':
        filtered_df = filtered_df[filtered_df['Gender'] == gender]
    
    if city != 'All':
        filtered_df = filtered_df[filtered_df['City'] == city]
    
    return filtered_df

# Callbacks pour mettre à jour les indicateurs et les graphiques
@app.callback(
    [Output('total-amount', 'children'),
     Output('total-purchases', 'children'),
     Output('histogram-chart', 'figure'),
     Output('pie-chart', 'figure'),
     Output('line-chart', 'figure')],
    [Input('gender-filter', 'value'),
     Input('city-filter', 'value')]
)
def update_dashboard(gender, city):
    filtered_df = filter_data(gender, city)
    
    # Calcul des indicateurs
    total_amount = f"{filtered_df['Total'].sum():.2f} €"
    total_purchases = len(filtered_df)
    
    # Histogramme des montants totaux par sexe et ville
    hist_fig = px.histogram(
        filtered_df, 
        x='Total',
        color='Gender',
        barmode='group',
        facet_col='City' if city == 'All' else None,
        title="Répartition des montants totaux des achats"
    )
    hist_fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    
    # Diagramme circulaire des catégories de produits
    pie_fig = px.pie(
        filtered_df,
        names='Product line',
        title="Répartition des catégories de produits"
    )
    pie_fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    
    # Évolution des achats par semaine
    weekly_data = filtered_df.groupby(['Year_Week', 'City'])['Total'].sum().reset_index()
    weekly_data = weekly_data.sort_values('Year_Week')
    
    line_fig = px.line(
        weekly_data,
        x='Year_Week',
        y='Total',
        color='City',
        title="Évolution des achats par semaine"
    )
    line_fig.update_xaxes(title="Semaine")
    line_fig.update_yaxes(title="Montant total des achats")
    line_fig.update_layout(margin=dict(l=20, r=20, t=30, b=20))
    
    return total_amount, str(total_purchases), hist_fig, pie_fig, line_fig

# Lancer l'application
if __name__ == '__main__':
    app.run(debug=True, port=8050, jupyter_mode="external")

FileNotFoundError: [Errno 2] No such file or directory: './Projet/supermarket_sales.csv'